In [25]:
import json
from pathlib import Path
import os

import pandas as pd
import s3fs


def read_cluster_csv(file_path, endpoint_url='https://storage.budsc.midwest-datascience.com'):
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    return pd.read_csv(s3.open(file_path, mode='rb'))

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)

people_json = kv_data_dir.joinpath('people.json')
visited_json = kv_data_dir.joinpath('visited.json')
sites_json = kv_data_dir.joinpath('sites.json')
measurements_json = kv_data_dir.joinpath('measurements.json')

In [26]:
class KVDB(object):
    def __init__(self, db_path):
        self._db_path = Path(db_path)
        self._db = {}
        self._load_db()

    def _load_db(self):
        if self._db_path.exists():
            with open(self._db_path) as f:
                self._db = json.load(f)

    def get_value(self, key):
        return self._db.get(key)

    def set_value(self, key, value):
        self._db[key] = value

    def save(self):
        with open(self._db_path, 'w') as f:
            json.dump(self._db, f, indent=2)

In [27]:
# I am going to really try to do this stuff locally. We shall see how it goes.
# Changed from read cluster to read csv
# I also got the data locally from the data folder

# Here I opted to roll with the primary key for my grouping
def create_sites_kvdb():
    db = KVDB(sites_json)
    # df = read_cluster_csv('data/external/tidynomicon/site.csv')
    df = pd.read_csv('/home/totennacht/Bellevue/dsc650/data/external/tidynomicon/site.csv')
    for site_id, group_df in df.groupby('site_id'):
        db.set_value(site_id, group_df.to_dict(orient='records')[0])
    db.save()


# This one was a bit arbitrary person_id seemed to be their last name but it really should have some kind of
# unique value for a primary key.
def create_people_kvdb():
    db = KVDB(people_json)
    # df = read_cluster_csv('data/external/tidynomicon/person.csv')
    df = pd.read_csv('/home/totennacht/Bellevue/dsc650/data/external/tidynomicon/person.csv')
    for person_id, group_df in df.groupby('person_id'):
        db.set_value(person_id, group_df.to_dict(orient='records')[0])
    db.save()


# I went with visit id looked like a primary key so I chose to group by it
def create_visits_kvdb():
    db = KVDB(visited_json)
    # df = read_cluster_csv('data/external/tidynomicon/visited.csv')
    df = pd.read_csv('/home/totennacht/Bellevue/dsc650/data/external/tidynomicon/visited.csv')
    for visit_id, group_df in df.groupby('visit_id'):
        db.set_value(visit_id, group_df.to_dict(orient='records')[0])
    db.save()


# here I could have gone with person_id or visit_id
def create_measurements_kvdb():
    db = KVDB(measurements_json)
    # df = read_cluster_csv('data/external/tidynomicon/measurements.csv')
    df = pd.read_csv('/home/totennacht/Bellevue/dsc650/data/external/tidynomicon/measurements.csv')
    for person_id, group_df in df.groupby('person_id'):
        db.set_value(person_id, group_df.to_dict(orient='records')[0])
    db.save()

In [28]:
# not bad I got it. I can see why you prefer to have people do things on the managed server. Hark work pays off. :)
create_sites_kvdb()
create_people_kvdb()
create_visits_kvdb()
create_measurements_kvdb()